# Mobile Price Calculation using scikit-learn custom script in Sagemaker # 

## Import necessary libraries ##

In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd


/Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/15/24 22:10:00] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=368070;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=276160;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/shitleshbakshi/Library/Application Support/sagemaker/config.yaml


In [35]:
sm_boto3 = boto3.client('sagemaker')
session = sagemaker.Session()
region = session.boto_session.region_name
bucket = 'mymob-demo-bucket' # Bucket created in AWS S3
print('Using Bucket ' + bucket)

Using Bucket mymob-demo-bucket


In [36]:
df = pd.read_csv('mob_price_classification_train.csv')

In [37]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [38]:
df.shape

(2000, 21)

## Feature Engineering

In [39]:
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [40]:
df.isnull().mean()*100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [41]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [42]:
label = features.pop(-1)
label

'price_range'

In [43]:
x = df[features]
y = df[label]

In [44]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [45]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [46]:
x.shape

(2000, 20)

In [47]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [48]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [49]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1600, 20) (400, 20) (1600,) (400,)


In [50]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [51]:
print(trainX.shape, testX.shape)

(1600, 21) (400, 21)


In [52]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
582,1232,0,2.9,1,1,1,24,0.3,169,5,...,361,809,1257,16,10,16,1,0,0,0
159,1840,0,0.5,1,12,0,34,0.7,142,1,...,311,1545,1078,8,0,10,0,0,0,1
1827,1692,0,2.1,0,4,1,2,0.9,106,1,...,1899,1904,3779,9,3,7,1,1,1,3
318,508,0,0.8,0,7,1,42,0.3,94,1,...,39,557,663,13,12,7,1,0,0,0
708,977,1,2.8,1,2,0,35,0.6,165,2,...,1502,1862,3714,19,3,10,0,1,1,3


In [53]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [54]:
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [55]:
trainX.to_csv('trainX_v1.csv', index=False)
testX.to_csv('testX_v1.csv', index=False)

## Push data to S3 ##

In [56]:
sk_prefix = 'sagemaker/mobile-price-classification-model/sklearncontainer'
trainpath = session.upload_data(path = 'trainX_v1.csv', bucket = bucket, key_prefix=sk_prefix)
testpath = session.upload_data(path = 'testX_v1.csv', bucket = bucket, key_prefix=sk_prefix)

In [57]:
print(trainpath, testpath)

s3://mymob-demo-bucket/sagemaker/mobile-price-classification-model/sklearncontainer/trainX_v1.csv s3://mymob-demo-bucket/sagemaker/mobile-price-classification-model/sklearncontainer/testX_v1.csv


In [58]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
    classifier = joblib.load(os.path.join(model_dir,'model.joblib'))
    return classifier

if __name__ =='__main__':
    
    print('[INFO] Extracting arguments')
    parser = argparse.ArgumentParser()
    
    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="trainX_v1.csv")
    parser.add_argument("--test-file", type=str, default="testX_v1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Overwriting script.py


In [59]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::116981766627:role/SagemakerRole",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [60]:
#launch training job with async call
sklearn_estimator.fit({'train': trainpath, 'test': testpath}, wait=True)

[12/16/24 23:26:17] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=937904;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=924613;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[12/16/24 23:26:19] INFO     Creating training-job with name:                                       ]8;id=545128;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=52282;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\
                             RF-custom-sklearn-2024-12-16-23-26-17-291                                             

2024-12-16 23:26:21 Starting - Starting the training job...
2024-12-16 23:26:35 Starting - Preparing the instances for training...
2024-12-16 23:27:28 Downloading - Downloading the training image.....2024-12-16 23:28:13,464 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-12-16 23:28:13,468 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-16 23:28:13,508 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-12-16 23:28:13,658 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-16 23:28:13,670 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-16 23:28:13,682 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-16 23:28:13,690 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_di

In [61]:
sklearn_estimator.latest_training_job.wait(logs='None')
artifact=sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)['ModelArtifacts']['S3ModelArtifacts']
print(artifact)


2024-12-16 23:28:32 Starting - Preparing the instances for training
2024-12-16 23:28:32 Downloading - Downloading the training image
2024-12-16 23:28:32 Training - Training image download completed. Training in progress.
2024-12-16 23:28:32 Uploading - Uploading generated training model
2024-12-16 23:28:32 Completed - Training job completed
s3://sagemaker-us-east-1-116981766627/RF-custom-sklearn-2024-12-16-23-26-17-291/output/model.tar.gz


In [62]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role="arn:aws:iam::116981766627:role/SagemakerRole",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [63]:
model_name

'Custom-sklearn-model-2024-12-16-23-48-30'

In [64]:
model

In [65]:
##Endpoints deployment
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2024-12-16-23-49-56


[12/16/24 23:49:59] INFO     Creating model with name: Custom-sklearn-model-2024-12-16-23-48-30     ]8;id=112269;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=349308;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\

[12/16/24 23:50:00] INFO     Creating endpoint-config with name                                     ]8;id=254671;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=285775;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/session.py#5889\5889]8;;\
                             Custom-sklearn-model-2024-12-16-23-49-56                                              

                    INFO     Creating endpoint with name Custom-sklearn-model-2024-12-16-23-49-56   ]8;id=126046;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=496181;file:///Users/shitleshbakshi/SagemakerTraining/.venv/lib/python3.12/site-packages/sagemaker/session.py#4711\4711]8;;\

-----!

In [66]:
endpoint_name

'Custom-sklearn-model-2024-12-16-23-49-56'

In [67]:
testX[features][0:2].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0]]

In [68]:
print(predictor.predict(testX[features][0:2].values.tolist()))

[3 0]


In [69]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '07dc54be-7161-4e1f-8a68-8344ec1bb1c8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '07dc54be-7161-4e1f-8a68-8344ec1bb1c8',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 16 Dec 2024 23:54:13 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}